# Preprocessing Pipeline

## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

## Functions

In [2]:
def remove_outliers(df, group_by="TaskName", value_col="price"):
    def filter_group(group):
        Q1 = group[value_col].quantile(0.25)
        Q3 = group[value_col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        return group[(group[value_col] >= lower) & (group[value_col] <= upper)]
    
    return df.groupby(group_by, group_keys=False).apply(filter_group)

In [3]:
def drop_rare(df, column, threshold, df_name="DataFrame"):
    print(f"----Dropping Rare {column}s from {df_name}----")
    print(f"{column}s before: {len(df[column].unique())}")
    counts = df[column].value_counts()
    rare_values = counts[counts < threshold].index
    df = df[~df[column].isin(rare_values)].copy()
    print(f"{column}s after: {len(df[column].unique())}")
    return df

## Read in Data

In [4]:
# CPI for price adjustment
cpi_index = pd.read_csv('data/cpi_data.csv')
cpi_index.head()

,Quarter,CPI
0,2021Q1,117.9
1,2021Q2,118.8
2,2021Q3,119.7
3,2021Q4,121.3
4,2022Q1,123.9


In [5]:
# Booking/Ticket data
df = pd.read_csv('data/ticket_data.csv', encoding = 'windows-1252')
df.head()

,ï»¿FCID,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,BOdoNum,BOdoText,cVMake,cVMakeModel,cVYear,idFuel,idLitres,idTransmission,idDrive,idIsHybrid,VMid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted
0,4,2172785,19/02/2025,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,"105,291 km",TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,TOY38983,16764,2500,NSW,Wollongong,2,0,0,0
1,4,2172921,19/02/2025,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,"Prescribed - 60,000 kilometres / 24 months",939.50,19957.0,"19,957 km",ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,NaN,16955,2790,NSW,South Eastern,2,0,0,0
2,9,2173119,19/02/2025,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,"Logbook Service - 135,000 km / 108 months",371.82,141320.0,"141,320 km",HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,HYU48327,20684,5554,SA,Yorke Peninsula,2,0,0,0
3,4,2173187,19/02/2025,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,"227,817 km",ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,ISU30317,2108,2208,NSW,Sydney,1,0,0,0
4,2,2173202,19/02/2025,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,"14,528 km",SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,SUB47068,17961,3844,VIC,Traralgon,2,0,0,0


## Rename Columns

In [6]:
df = df.rename(columns={
    'cVMake': 'Make',
    'cVMakeModel': 'Model',
    'cVYear': 'Year',
    'idFuel': 'FuelType',
    'idLitres': 'EngineSize',
    'idTransmission': 'Transmission',
    'idDrive': 'DriveType',
    'idIsHybrid': 'IsHybrid',
    'BOdoNum' : 'Odometer',
    })

## Drop Columns that will not be used

In [7]:
columns_to_drop = ['ï»¿FCID', 'BOdoText', 'VMid']
df.drop(columns = columns_to_drop, inplace=True)
df.head()

,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted
0,2172785,19/02/2025,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,16764,2500,NSW,Wollongong,2,0,0,0
1,2172921,19/02/2025,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,"Prescribed - 60,000 kilometres / 24 months",939.50,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,16955,2790,NSW,South Eastern,2,0,0,0
2,2173119,19/02/2025,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,"Logbook Service - 135,000 km / 108 months",371.82,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,20684,5554,SA,Yorke Peninsula,2,0,0,0
3,2173187,19/02/2025,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,2108,2208,NSW,Sydney,1,0,0,0
4,2173202,19/02/2025,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,17961,3844,VIC,Traralgon,2,0,0,0


## Remove Rows


In [8]:
# Remove custom services and repairs
df = df[df['IsCustomService'] != '1']
df = df[df['IsCustomRepair'] != '1']
df = df[df['TaskName'] != 'Custom Repair']

# Remove products and tyres as they are unpredictable without quantity information 
df = df[df['TaskName'] != '((Products))']
df = df[df['TaskName'] != '((Tyres))']
df = df[df['TaskName'] != 'Tyre Replacement']

# Remove Rare ticket types
df = df[df['BTicketType'] != 'OtherTicket']
df = df[df['BTicketType'] != 'Custom']
df = df[df['BTicketType'] != 'Basic']

# Remove Tickets with zero or negative prices 
df = df[df['PriceIncGSTRaw'] > 0]

## Create Months and Kilometer Columns from TicketType

In [9]:
# Create Columns
df['Distance'] = None
df['Months'] = None

mask = df['BTicketType'] == 'Log'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Rest'] = split_1[1].fillna('')

split_2 = df.loc[mask, 'Rest'].str.split(' / ', n=1, expand=True)
df.loc[mask, 'Distance'] = split_2[0]
df.loc[mask, 'Months'] = split_2[1]

mask = df['BTicketType'] == 'Capped'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Distance'] = split_1[1].fillna('')

mask = df['BTicketType'] == 'Prescribed'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Distance'] = split_1[1].fillna('')

df = df.drop(columns=['Rest'])

In [10]:
# Format as Numeric
df['Months'] = df['Months'].astype(str).str.extract(r'(\d+)')
df['Months'] = pd.to_numeric(df['Months'], errors='coerce')

df['Distance'] = df['Distance'].astype(str).str.replace(',', '', regex=False)  
df['Distance'] = df['Distance'].astype(str).str.extract(r'(\d+)')
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')

df.head()

,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted,Distance,Months
0,2172785,19/02/2025,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,16764,2500,NSW,Wollongong,2,0,0,0,NaN,NaN
1,2172921,19/02/2025,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,Prescribed,939.50,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,16955,2790,NSW,South Eastern,2,0,0,0,60000.0,NaN
2,2173119,19/02/2025,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,Logbook Service,371.82,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,20684,5554,SA,Yorke Peninsula,2,0,0,0,135000.0,108.0
3,2173187,19/02/2025,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,2108,2208,NSW,Sydney,1,0,0,0,NaN,NaN
4,2173202,19/02/2025,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,17961,3844,VIC,Traralgon,2,0,0,0,NaN,NaN


## Create Adjusted Price based on CPI

In [11]:
# 1. Create date column
df['BCreatedDateAEST'] = pd.to_datetime(df['BCreatedDateAEST'], format='mixed', dayfirst=True);
df['Date'] = df['BCreatedDateAEST'].dt.date

In [12]:
# 2. Price Adjustment
df = df[df['PriceIncGSTRaw'] != 0]
df['Date'] = pd.to_datetime(df['Date']) # ensure 'date' is in datetime format
df['Quarter'] = df['Date'].dt.to_period('Q').astype(str)
df = df.merge(cpi_index, on='Quarter', how='left')
base_cpi = cpi_index['CPI'].iloc[-1]
df['AdjustedPrice'] = round(df['PriceIncGSTRaw'] * (base_cpi / df['CPI']), 2) # round to 2 decimal places for consistency
df['AdjustedPrice'] = round(df['AdjustedPrice'].fillna(df['PriceIncGSTRaw'])) # use original price where CPI is missing (for current quarter which does not need adjustment)

In [13]:
df.head()

,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted,Distance,Months,Date,Quarter,CPI,AdjustedPrice
0,2172785,2025-02-19,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,16764,2500,NSW,Wollongong,2,0,0,0,NaN,NaN,2025-02-19,2025Q1,140.7,665.0
1,2172921,2025-02-19,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,Prescribed,939.50,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,16955,2790,NSW,South Eastern,2,0,0,0,60000.0,NaN,2025-02-19,2025Q1,140.7,940.0
2,2173119,2025-02-19,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,Logbook Service,371.82,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,20684,5554,SA,Yorke Peninsula,2,0,0,0,135000.0,108.0,2025-02-19,2025Q1,140.7,372.0
3,2173187,2025-02-19,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,2108,2208,NSW,Sydney,1,0,0,0,NaN,NaN,2025-02-19,2025Q1,140.7,51.0
4,2173202,2025-02-19,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,17961,3844,VIC,Traralgon,2,0,0,0,NaN,NaN,2025-02-19,2025Q1,140.7,32.0


## Remove Columns that will not be used

In [14]:
columns_to_drop = [ 'BCreatedDateAEST', 'CPI', 'Quarter', 'PriceIncGSTRaw' , 'BShopID', 'BShopRegionName', 'BShopPostcode', 'IsDeleted', 'BStatusFromDateTimeAEST', 'Date', 'BStatusFinal', ]
df.drop(columns = columns_to_drop, inplace=True)
df.head()

,BookingID,BStatusAfterSubmitted,BTicketID,BTicketType,TaskName,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopState,BShopRegionClass,IsCustomService,IsCustomRepair,Distance,Months,AdjustedPrice
0,2172785,33. Approved,3647415,Repair,Tail Light Assembly Replacement,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,NSW,2,0,0,NaN,NaN,665.0
1,2172921,33. Approved,3647828,Prescribed,Prescribed,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,NSW,2,0,0,60000.0,NaN,940.0
2,2173119,33. Approved,3648072,Log,Logbook Service,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,SA,2,0,0,135000.0,108.0,372.0
3,2173187,33. Approved,3648235,Repair,Roadworthy Inspection / Pink Slips,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,NSW,1,0,0,NaN,NaN,51.0
4,2173202,33. Approved,3648252,Repair,Balance Tyres,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,VIC,2,0,0,NaN,NaN,32.0


## Remove Duplicate Rows

In [15]:
# Duplicates can skew analysis and lead to incorrect conclusions, thus it is important each row is unique. Duplicates in this dataset are considered as rows that contain the same values across all columns other than the ID columns 
print(f"size before: {df.shape}" )
df = df.drop_duplicates(subset=[col for col in df.columns if col not in ['BookingID', 'BTicketID', 'BCreatedDateAEST', 'BStatusFromDateTimeAEST']])
print(f"size after: {df.shape}" )

size before: (935268, 21)
size after: (841178, 21)


## Remove False Negatives 

In [16]:
# False negatives are rows which have a status of '16. Requires Changes' when they should have a status of '33. Approved'. This occurs in the dataset as bookings contain multiple tickets. If a single ticket in a booking requires changes, the entire booking is marked as 'Requires Changes', thus marking tickets that do not require changes incorrectly. These false negatives can be detected and removed by checking if a duplicate entry exists where only the status changes.
print(f"size before: {df.shape}" )
df['StatusPriority'] = df['BStatusAfterSubmitted'].apply(lambda x: 0 if x == '33. Approved' else 1) # Assign priority: approved gets highest priority (lowest number)
dedup_cols = [col for col in df.columns if col not in ['BStatusAfterSubmitted', 'StatusPriority','BCreatedDateAEST', 'BStatusFromDateTimeAEST', 'Date' ]] # Define columns to check for duplicates
df = df.sort_values(by=dedup_cols + ['StatusPriority']) # Sort so approved status is first
df = df.drop_duplicates(subset=dedup_cols, keep='first') # Keep the first occurrence (which is the approved status)
df = df.drop(columns='StatusPriority') # drop the temporary column used for sorting

df['Label'] = df['BStatusAfterSubmitted'].map({'33. Approved': 1, '16. Requires Changes': 0, '29. Rejected': 0}) # create label column for model training
df = df.drop(columns=['BStatusAfterSubmitted'], axis=1) # drop the original status column as it is no longer needed
print(f"size after: {df.shape}" ) 

size before: (841178, 21)
size after: (729978, 21)


## Remove Rejected Cases

In [17]:
df = df[df['Label'] == 1];

## Fill in missing distances with odometer

In [18]:
df["Distance"] = df["Distance"].fillna(df["Odometer"])

## Final Column Removal

In [19]:

columns_to_drop = ['BookingID', 'BTicketID', 'IsCustomService', 'IsCustomRepair', 'BShopRegionClass', 'BShopState', 'IsHybrid', 'Label', 'Odometer']
df.drop(columns = columns_to_drop, inplace=True)
df.head()

,BTicketType,TaskName,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,Distance,Months,AdjustedPrice
339544,Log,Logbook Service,SKODA,SKODA FABIA III,2017,Petrol,1.0,Auto,NaN,75000.0,60.0,710.0
334397,Log,Logbook Service,MAZDA,MAZDA CX-9,2018,Petrol,2.5,Auto,2WD,80000.0,96.0,586.0
339546,Capped,Capped Price Service,HYUNDAI,HYUNDAI KONA,2019,Petrol,1.6,Auto,4WD,40000.0,NaN,354.0
338427,Capped,Capped Price Service,HYUNDAI,HYUNDAI TUCSON,2019,Diesel,2.0,Auto,4WD,90000.0,NaN,499.0
338426,Repair,Park Light Globe Replacement,HYUNDAI,HYUNDAI TUCSON,2019,Diesel,2.0,Auto,4WD,95200.0,NaN,7.0


## Split data Frame by Ticket Type

In [20]:
df_repair = df[df['BTicketType'] == 'Repair'].drop(columns=['BTicketType'])
df_log = df[df['BTicketType'] == 'Log'].drop(columns=['BTicketType'])
df_capped = df[df['BTicketType'] == 'Capped'].drop(columns=['BTicketType'])
df_prescribed = df[df['BTicketType'] == 'Prescribed'].drop(columns=['BTicketType'])

## Dropping Rare Cases

In [21]:
df_repair = drop_rare(df_repair, 'TaskName', 100, df_name="Repair DF")
df_log = drop_rare(df_log, 'Model', 20, df_name="Log DF")
df_capped = drop_rare(df_capped, 'Model', 20, df_name="Capped DF")
df_prescribed = drop_rare(df_prescribed, 'Model', 20, df_name="Prescribed DF")

----Dropping Rare TaskNames from Repair DF----
TaskNames before: 1579
TaskNames after: 347
----Dropping Rare Models from Log DF----
Models before: 341
Models after: 195
----Dropping Rare Models from Capped DF----
Models before: 557
Models after: 226
----Dropping Rare Models from Prescribed DF----
Models before: 227
Models after: 46


In [22]:
import os

# Define paths
paths = {
    "repair": "../backend/data/preprocessed_repair_data.csv",
    "log": "../backend/data/preprocessed_log_data.csv",
    "capped": "../backend/data/preprocessed_capped_data.csv",
    "prescribed": "../backend/data/preprocessed_prescribed_data.csv",
}

# Make sure the directories exist
for path in paths.values():
    dir_path = os.path.dirname(path)
    os.makedirs(dir_path, exist_ok=True)  

# Save CSVs
df_repair.to_csv(paths["repair"], index=False)
df_log.to_csv(paths["log"], index=False)
df_capped.to_csv(paths["capped"], index=False)
df_prescribed.to_csv(paths["prescribed"], index=False)

print("All files saved successfully:")
for name, path in paths.items():
    print(f"{name}: {os.path.abspath(path)}")

All files saved successfully:
repair: /home/dylan/git/CAPSTONE_T316/backend/data/preprocessed_repair_data.csv
log: /home/dylan/git/CAPSTONE_T316/backend/data/preprocessed_log_data.csv
capped: /home/dylan/git/CAPSTONE_T316/backend/data/preprocessed_capped_data.csv
prescribed: /home/dylan/git/CAPSTONE_T316/backend/data/preprocessed_prescribed_data.csv
